In [11]:
# Librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim
import torch.utils
from torch.utils.data import random_split, DataLoader
import torch.utils.tensorboard

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST, SVHN, USPS

from tqdm import tqdm
import os
import time
import PIL

In [34]:
# ---------------- Dataset Loader ------------------------
def mnist_dataloader(to_rgb=True, train=True, batch_size=64):
    trans = transforms.Compose([
        #PIL.Image.convert('RGB'),
        transforms.Resize(size=(32,32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)), 
    ])
    mnist_dataset = MNIST('./data/MNIST', download=True, transform=trans)
    loader = DataLoader(mnist_dataset, batch_size=batch_size, shuffle=True)
    return loader

def svhn_dataloader(train=True, batch_size=64):
    trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5), (0.5, 0.5))
    ])
    dataset = SVHN('./data/SVHN', download=True, transform=trans, split=('train' if train else 'test'))
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

def usps_loader(train=True, batch_size=64):
    trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5), (0.5,0.5))
    ])
    dataset = USPS('./data/USPS', download=True, transform=trans, train=train)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

In [19]:
# ----------------- Functions for F-CADA --------------------- #
def sample_data(mnist=True, svhn=False, usps=False):
    if mnist: dataset = mnist_dataloader()
    elif svhn: dataset = svhn_dataloader()
    elif usps: dataset = usps_loader()
    n=len(dataset)

    X=torch.Tensor(n,1,28,28)
    Y=torch.LongTensor(n)

    inds=torch.randperm(len(dataset))
    for i,index in enumerate(inds):
        x,y=dataset[index]
        X[i]=x
        Y[i]=y
    return X,Y

def create_target_samples(n=1, mnist=False, svhn=True, usps=False):
    if mnist: dataset = mnist_dataloader()
    elif svhn: dataset = svhn_dataloader()
    elif usps: dataset = usps_loader()
    X,Y=[],[]
    classes=10*[n]

    i=0
    while True:
        if len(X)==n*10:
            break
        x,y=dataset[i]
        if classes[y]>0:
            X.append(x)
            Y.append(y)
            classes[y]-=1
        i+=1

    assert (len(X)==n*10)
    return torch.stack(X,dim=0),torch.from_numpy(np.array(Y))

"""
G1: a pair of pic comes from same domain ,same class
G3: a pair of pic comes from same domain, different classes

G2: a pair of pic comes from different domain,same class
G4: a pair of pic comes from different domain, different classes
"""
def create_groups(X_s,Y_s,X_t,Y_t,seed=1):
    #change seed so every time wo get group data will different in source domain,but in target domain, data not change
    torch.manual_seed(1 + seed)
    torch.cuda.manual_seed(1 + seed)
    n=X_t.shape[0] #10*shot
    #shuffle order
    classes = torch.unique(Y_t)
    classes=classes[torch.randperm(len(classes))]
    class_num=classes.shape[0]
    shot=n//class_num

    def s_idxs(c):
        idx=torch.nonzero(Y_s.eq(int(c)))
        return idx[torch.randperm(len(idx))][:shot*2].squeeze()
    def t_idxs(c):
        return torch.nonzero(Y_t.eq(int(c)))[:shot].squeeze()
    
    source_idxs = list(map(s_idxs, classes))
    target_idxs = list(map(t_idxs, classes))

    source_matrix=torch.stack(source_idxs)
    target_matrix=torch.stack(target_idxs)

    G1, G2, G3, G4 = [], [] , [] , []
    Y1, Y2 , Y3 , Y4 = [], [] ,[] ,[]
    for i in range(10):
        for j in range(shot):
            G1.append((X_s[source_matrix[i][j*2]],X_s[source_matrix[i][j*2+1]]))
            Y1.append((Y_s[source_matrix[i][j*2]],Y_s[source_matrix[i][j*2+1]]))
            G2.append((X_s[source_matrix[i][j]],X_t[target_matrix[i][j]]))
            Y2.append((Y_s[source_matrix[i][j]],Y_t[target_matrix[i][j]]))
            G3.append((X_s[source_matrix[i%10][j]],X_s[source_matrix[(i+1)%10][j]]))
            Y3.append((Y_s[source_matrix[i % 10][j]], Y_s[source_matrix[(i + 1) % 10][j]]))
            G4.append((X_s[source_matrix[i%10][j]],X_t[target_matrix[(i+1)%10][j]]))
            Y4.append((Y_s[source_matrix[i % 10][j]], Y_t[target_matrix[(i + 1) % 10][j]]))

    groups=[G1,G2,G3,G4]
    groups_y=[Y1,Y2,Y3,Y4]
    #make sure we sampled enough samples
    for g in groups:
        assert(len(g)==n)
    return groups,groups_y

def sample_groups(X_s,Y_s,X_t,Y_t,seed=1):


    print("Sampling groups")
    return create_groups(X_s,Y_s,X_t,Y_t,seed=seed)

In [22]:
# ------------------------ Functions for CCSA ------------------------------ #
import random
import os

from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Input, Lambda, Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD, RMSprop, Adam, Adadelta, Nadam
from keras import backend as K
import numpy   as np
import sys

def printn(string):
    sys.stdout.write(string)
    sys.stdout.flush()
    
# Re configurar create_pairs para aceptar todos los datasets -> transformar todo para pytorch??
def Create_Pairs(domain_adaptation_task,repetition,sample_per_class):
    UM  = domain_adaptation_task
    cc  = repetition
    SpC = sample_per_class

    if UM != 'MNIST_to_USPS':
        if UM != 'USPS_to_MNIST':
            raise Exception('domain_adaptation_task should be either MNIST_to_USPS or USPS_to_MNIST')
    if cc <0 or cc>10:
        raise Exception('number of repetition should be between 0 and 9.')
    if SpC <1 or SpC>7:
            raise Exception('number of sample_per_class should be between 1 and 7.')

    print 'Creating pairs for repetition: '+str(cc)+' and sample_per_class: '+str(sample_per_class)
    X_train_target=np.load('./row_data/' + UM + '_X_train_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')
    y_train_target=np.load('./row_data/' + UM + '_y_train_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')
    X_train_source=np.load('./row_data/' + UM + '_X_train_source_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')
    y_train_source=np.load('./row_data/' + UM + '_y_train_source_repetition_' + str(cc) + '_sample_per_class_' + str(SpC) + '.npy')

    Training_P=[]
    Training_N=[]
    for trs in range(len(y_train_source)):
        for trt in range(len(y_train_target)):
            if y_train_source[trs]==y_train_target[trt]:
                Training_P.append([trs,trt])
            else:
                Training_N.append([trs,trt])

    random.shuffle(Training_N)
    Training = Training_P+Training_N[:3*len(Training_P)]
    random.shuffle(Training)


    X1=np.zeros([len(Training),16,16],dtype='float32')
    X2=np.zeros([len(Training),16,16],dtype='float32')

    y1=np.zeros([len(Training)])
    y2=np.zeros([len(Training)])
    yc=np.zeros([len(Training)])

    for i in range(len(Training)):
        in1,in2=Training[i]
        X1[i,:,:]=X_train_source[in1,:,:]
        X2[i,:,:]=X_train_target[in2,:,:]

        y1[i]=y_train_source[in1]
        y2[i]=y_train_target[in2]
        if y_train_source[in1]==y_train_target[in2]:
            yc[i]=1

    if not os.path.exists('./pairs'):
        os.makedirs('./pairs')

    np.save('./pairs/' + UM + '_X1_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy', X1)
    np.save('./pairs/' + UM + '_X2_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy', X2)

    np.save('./pairs/' + UM + '_y1_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy', y1)
    np.save('./pairs/' + UM + '_y2_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy', y2)
    np.save('./pairs/' + UM + '_yc_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy', yc)
    
def Create_Model():
    img_rows, img_cols = 16, 16
    nb_filters = 32
    pool_size = (2, 2)
    kernel_size = (3, 3)
    input_shape = (img_rows, img_cols, 1)
    # Model
    model = Sequential()
    model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]),
                            padding ='valid',
                            input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1])))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))
    # FC
    model.add(Flatten())
    model.add(Dense(120))
    model.add(Activation('relu'))
    model.add(Dense(84))
    model.add(Activation('relu'))
    return model

def euclidean_distance(vects):
    eps = 1e-08
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), eps))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def training_the_model(model,domain_adaptation_task,repetition,sample_per_class):
    nb_classes=10
    UM = domain_adaptation_task
    cc = repetition
    SpC = sample_per_class

    if UM != 'MNIST_to_USPS':
        if UM != 'USPS_to_MNIST':
            raise Exception('domain_adaptation_task should be either MNIST_to_USPS or USPS_to_MNIST')
    if cc < 0 or cc > 10:
        raise Exception('number of repetition should be between 0 and 9.')
    if SpC < 1 or SpC > 7:
        raise Exception('number of sample_per_class should be between 1 and 7.')
    epoch = 80  # Epoch number
    batch_size = 256
    X_test = np.load('./row_data/' + UM + '_X_test_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC)+'.npy')
    y_test = np.load('./row_data/' + UM + '_y_test_target_repetition_' + str(cc) + '_sample_per_class_' + str(SpC)+'.npy')
    X_test = X_test.reshape(X_test.shape[0], 16, 16, 1)
    y_test = np_utils.to_categorical(y_test, nb_classes)

    X1 = np.load('./pairs/' + UM + '_X1_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy')
    X2 = np.load('./pairs/' + UM + '_X2_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy')

    X1 = X1.reshape(X1.shape[0], 16, 16, 1)
    X2 = X2.reshape(X2.shape[0], 16, 16, 1)

    y1 = np.load('./pairs/' + UM + '_y1_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy')
    y2 = np.load('./pairs/' + UM + '_y2_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy')
    yc = np.load('./pairs/' + UM + '_yc_count_' + str(cc) + '_SpC_' + str(SpC) + '.npy')

    y1 = np_utils.to_categorical(y1, nb_classes)
    y2 = np_utils.to_categorical(y2, nb_classes)

    print('Training the model - Epoch '+str(epoch))
    nn=batch_size
    best_Acc = 0
    for e in range(epoch):
        if e % 10 == 0:
            printn(str(e) + '->')
        for i in range(len(y2) / nn):
            loss = model.train_on_batch([X1[i * nn:(i + 1) * nn, :, :, :], X2[i * nn:(i + 1) * nn, :, :, :]],
                                        [y1[i * nn:(i + 1) * nn, :], yc[i * nn:(i + 1) * nn, ]])
            loss = model.train_on_batch([X2[i * nn:(i + 1) * nn, :, :, :], X1[i * nn:(i + 1) * nn, :, :, :]],
                                        [y2[i * nn:(i + 1) * nn, :], yc[i * nn:(i + 1) * nn, ]])

        Out = model.predict([X_test, X_test])
        Acc_v = np.argmax(Out[0], axis=1) - np.argmax(y_test, axis=1)
        Acc = (len(Acc_v) - np.count_nonzero(Acc_v) + .0000001) / len(Acc_v)

        if best_Acc < Acc:
            best_Acc = Acc
    print str(e)
    return best_Acc

SyntaxError: invalid syntax (<ipython-input-22-acdec652788f>, line 32)

In [ ]:
# -------------------------- Functions for MME* -------------------- #


In [35]:
mnist = MNIST_dataloader(True, True)
svhn = svhn_dataloader()
#usps = usps_loader()

Using downloaded and verified file: ./data/SVHN\train_32x32.mat


In [38]:
for a in mnist:
    print(a)
    break

RuntimeError: output with shape [1, 32, 32] doesn't match the broadcast shape [2, 32, 32]